# MODIFIED: 24:12 36 hour prediction window

In this notebook, we will be modifying the 48 hour prediction window to be a 36 hour prediction window. Using the data from each 24 hour time chunk, we will predict whether or not a patient will get AKI in the next 12 hours- WHICH MODIFIES THE CURRENT ML ALGORITHM. Hopefully, I'll be able to get it such that the doctor can just input whatever settings they want, and this notebook will update accordingly.

Training and testing sets are separated via patients' id. We use a .npy file found in the Data folder that contains JUST patient IDs, so when we train on the actual dataset, the split is already clear. In this case, Zijian has already split the data and saved it into train.pkl and test.pkl.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
hadm_id = np.load('C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/all_ids.npy')
# randomly spliated 70-30
np.random.seed(123) # set seed for reproducibility 
train_ids = np.random.choice( hadm_id, round(len(hadm_id)*0.7), replace = False )
test_ids = np.array(list(set(hadm_id) - set(train_ids)))

print("all patient ids stored in 'HADM_ID'", hadm_id, 
      "; the total number of patients being 4120, \n which is 147 short from the 4267 supposed patients stated in the article :(")

all patient ids stored in 'HADM_ID' [20001687 20005241 20006999 ... 29994991 29997844 29998115] ; the total number of patients being 4120, 
 which is 147 short from the 4267 supposed patients stated in the article :(


In [3]:
print("length of training data:", len(train_ids), '\n', "length of testing data:", len(test_ids))

length of training data: 2884 
 length of testing data: 1236


### Revelations

Data -> byID: all 2000+ hadm_ids containing each patient's hourly data. 

Data -> byID_daily: all 2000+ hadm_ids DAILY summaries of their data (basically all of the daily minimums)

In [ ]:
# prep test code for actual code below
temp = pd.read_pickle('C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID/20955149.pkl')
print(temp)

row_count=0

temp_df= pd.DateFrame()

for id in train_ids:
    for row in id:
        row_count+=1

    rows_to_process= temp[0:12]
    min_values = np.amin(rows_to_process, axis=0)

In [304]:
hourly = pd.read_pickle('C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID/20955149.pkl')
hourly.head()
hourly_df= pd.DataFrame(hourly)

non_numeric_columns = []

for col in hourly.columns:
    if hourly[col].dtype == object:  # Check if column contains objects (usually strings)
        non_numeric_columns.append(col)



#WITHOUT NUMBRES: timedelta, Heart Rhythm, gender, admission_category


hourly_compounds= hourly.iloc[:, 2:550] 
compounds_and_occurences= hourly.iloc[:, 2:576]

#calculate and min values of compounds and max values of compounds + occurences and concatenate them together
compounds_grouped_mins= hourly_compounds.drop(columns= ['Heart Rhythm']).groupby(hourly_compounds.index // 12).min()
compounds_grouped_mins.columns = [col + "_min" for col in compounds_grouped_mins.columns]
grouped_maxs= compounds_and_occurences.drop(columns= ['Heart Rhythm', 'gender', 'admission_category']).groupby(compounds_and_occurences.index // 12).max()
grouped_maxs.columns = [col + "_max" for col in grouped_maxs.columns]
grouped = pd.concat([compounds_grouped_mins, grouped_maxs], axis=1)



#Add ID to the front of the list and gender between age and basline creatinine
id_values= [20955149]*len(grouped)
twelvehour_vitals = pd.DataFrame({'hadm_id': id_values, **grouped}) 
gender_values= [hourly['gender'][0]]*len(grouped)

twelvehour_vitals.insert(loc = 550,
          column = 'gender',
          value = gender_values)

#DIDN'T ADD FOR SOME REASON?!?!?!?



#ADD AKI in 12 hours to the list

len(twelvehour_vitals.columns)

1120

In [238]:
# prep test code for actual code below
daily = pd.read_pickle('C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID_daily/20955149.pkl')
dailydf= pd.DataFrame(daily)
pd.set_option('display.max_columns', None)  # None means unlimited columns displayed
daily

,hadm_id,arterial_bp_diastolic_min,arterial_bp_mean_min,arterial_bp_systolic_min,cvp_min,heart_rate_min,spo2_min,pap_diastolic_min,pap_mean_min,pap_systolic_min,cardiac_output_min,IABP_present_min,ECMO_present_min,crystalloid_min,colloid_min,enteral_intake_min,packed_red_blood_cells_min,blood_salvage_min,platelets_min,plasma_min,cryoprecipitate_min,urine_output_min,chest_tube_output_min,enteral_output_min,so2_min,po2_min,pco2_min,fio2_min,aado2_calc_min,pao2fio2ratio_min,ph_min,baseexcess_min,bicarbonate_min,totalco2_min,hematocrit_min,hemoglobin_min,chloride_min,calcium_min,potassium_min,sodium_min,lactate_min,glucose_min,mch_min,mchc_min,mcv_min,platelet_min,rbc_min,rdw_min,wbc_min,basophils_abs_min,eosinophils_abs_min,lymphocytes_abs_min,monocytes_abs_min,neutrophils_abs_min,basophils_min,eosinophils_min,lymphocytes_min,monocytes_min,neutrophils_min,bands_min,immature_granulocytes_min,metamyelocytes_min,nrbc_min,albumin_min,globulin_min,total_protein_min,aniongap_min,bun_min,creatinine_min,fibrinogen_min,thrombin_min,inr_min,pt_min,ptt_min,alt_min,alp_min,ast_min,amylase_min,bilirubin_total_min,bilirubin_direct_min,bilirubin_indirect_min,ck_cpk_min,ck_mb_min,ggt_min,ld_ldh_min,HIV_med_min,acetaminophen-IV_min,acetaminophen-PO_min,acetazolamide-IV_min,acetazolamide-PO_min,acetylcysteine-INHALATION_min,acetylcysteine-PO_min,acetylsalicylic acid_min,acyclovir-IV_min,acyclovir-PO_min,albuterol-INHALATION_min,albuterol-ipratropium_min,alendronate-PO_min,alfuzosin-PO_min,allopurinol-PO_min,alprazolam-PO_min,alteplase-IV_min,aluminium hydroxide-PO_min,aminocaproic acid-IV_min,amiodarone-IV_min,amiodarone-PO_min,amitriptyline-PO_min,amlodipine-PO_min,amoxicillin-PO_min,amoxicillin-clavulanate_min,amphetamine-dextroamphetamine_min,amphotericin b lipid complex (dmpc/dmpg)-IV_min,ampicillin-IV_min,ampicillin-PO_min,ampicillin-sulbactam_min,anastrozole-PO_min,angiotensin ii-IV_min,apixaban-PO_min,argatroban-IV_min,aripiprazole-PO_min,ascorbic acid-IV_min,ascorbic acid-PO_min,atazanavir-PO_min,atenolol-PO_min,atorvastatin-PO_min,atovaquone-PO_min,atropine-IV_min,azathioprine-PO_min,azithromycin-PO_min,aztreonam-IV_min,baclofen-PO_min,barium sulfate-PO_min,beclomethasone-INHALATION_min,benzonatate-PO_min,benztropine mesylate-PO_min,bethanechol-PO_min,bisacodyl-PO_min,bivalirudin-IV_min,budesonide-INHALATION_min,budesonide-PO_min,bumetanide-IV_min,bumetanide-PO_min,buprenorphine-SL_min,bupropion-PO_min,buspirone-PO_min,calcitriol-IV_min,calcitriol-PO_min,calcium chloride-IV_min,calcium gluconate-IV_min,calcium-IV_min,calcium-PO_min,captopril-PO_min,carbamazepine-PO_min,carbidopa-levodopa_min,carnitine-PO_min,carvedilol-PO_min,cefazolin-IV_min,cefepime-IV_min,ceftaroline-IV_min,ceftazidime-IV_min,ceftriaxone-IV_min,cephalexin-PO_min,cetirizine-PO_min,chlorothiazide-IV_min,chlorothiazide-PO_min,chlorpromazine-PO_min,chlorthalidone-PO_min,cholestyramine-PO_min,ciclosporin-PO_min,cilostazol-PO_min,cimetidine-PO_min,cinacalcet-PO_min,ciprofloxacin-IV_min,ciprofloxacin-PO_min,cisatracurium-IV_min,citalopram-PO_min,clarithromycin-PO_min,clevidipine-IV_min,clindamycin-IV_min,clindamycin-PO_min,clonazepam-PO_min,clonidine-PO_min,clopidogrel-PO_min,codeine-PO_min,colchicine-PO_min,colesevelam-PO_min,cosyntropin-IV_min,cyanocobalamin (vitamin b12)_min,cyclobenzaprine-PO_min,dabigatran etexilate-PO_min,danazol-PO_min,dapsone-PO_min,daptomycin-IV_min,darifenacin-PO_min,desmopressin-IV_min,desmopressin-PO_min,dexamethasone-IV_min,dexmedetomidine-IV_min,diatrizoate meglumine-PO_min,diazepam-PO_min,dicycloverine-PO_min,digoxin-IV_min,digoxin-PO_min,diltiazem-IV_min,diltiazem-PO_min,diphenhydramine-IV_min,diphenhydramine-PO_min,dipyridamole-PO_min,dobutamine-IV_min,docusate-PO_min,dofetilide-PO_min,dolutegravir-PO_min,donepezil-PO_min,dopamine-IV_min,doxazosin-PO_min,doxepin-PO_min,doxercalciferol-IV_min,doxycycline-IV_min,doxycycline-PO_min,dronabinol-PO_min,dronedarone-PO_min,duloxetine-PO_min,efavirenz-PO_min,enalapril-PO_min,enalaprilat-IV_min,enoxapa

In [240]:
hourlydf= pd.DataFrame(hourly)
hourlydf.head()


,hadm_id,timedelta,arterial_bp_diastolic,arterial_bp_mean,arterial_bp_systolic,cvp,heart_rate,spo2,pap_diastolic,pap_mean,pap_systolic,cardiac_output,Heart Rhythm,IABP_present,ECMO_present,crystalloid,colloid,enteral_intake,packed_red_blood_cells,blood_salvage,platelets,plasma,cryoprecipitate,urine_output,chest_tube_output,enteral_output,so2,po2,pco2,fio2,aado2_calc,pao2fio2ratio,ph,baseexcess,bicarbonate,totalco2,hematocrit,hemoglobin,chloride,calcium,potassium,sodium,lactate,glucose,mch,mchc,mcv,platelet,rbc,rdw,wbc,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,neutrophils_abs,basophils,eosinophils,lymphocytes,monocytes,neutrophils,bands,immature_granulocytes,metamyelocytes,nrbc,albumin,globulin,total_protein,aniongap,bun,creatinine,fibrinogen,thrombin,inr,pt,ptt,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,HIV_med,acetaminophen-IV,acetaminophen-PO,acetazolamide-IV,acetazolamide-PO,acetylcysteine-INHALATION,acetylcysteine-PO,acetylsalicylic acid,acyclovir-IV,acyclovir-PO,albuterol-INHALATION,albuterol-ipratropium,alendronate-PO,alfuzosin-PO,allopurinol-PO,alprazolam-PO,alteplase-IV,aluminium hydroxide-PO,aminocaproic acid-IV,amiodarone-IV,amiodarone-PO,amitriptyline-PO,amlodipine-PO,amoxicillin-PO,amoxicillin-clavulanate,amphetamine-dextroamphetamine,amphotericin b lipid complex (dmpc/dmpg)-IV,ampicillin-IV,ampicillin-PO,ampicillin-sulbactam,anastrozole-PO,angiotensin ii-IV,apixaban-PO,argatroban-IV,aripiprazole-PO,ascorbic acid-IV,ascorbic acid-PO,atazanavir-PO,atenolol-PO,atorvastatin-PO,atovaquone-PO,atropine-IV,azathioprine-PO,azithromycin-PO,aztreonam-IV,baclofen-PO,barium sulfate-PO,beclomethasone-INHALATION,benzonatate-PO,benztropine mesylate-PO,bethanechol-PO,bisacodyl-PO,bivalirudin-IV,budesonide-INHALATION,budesonide-PO,bumetanide-IV,bumetanide-PO,buprenorphine-SL,bupropion-PO,buspirone-PO,calcitriol-IV,calcitriol-PO,calcium chloride-IV,calcium gluconate-IV,calcium-IV,calcium-PO,captopril-PO,carbamazepine-PO,carbidopa-levodopa,carnitine-PO,carvedilol-PO,cefazolin-IV,cefepime-IV,ceftaroline-IV,ceftazidime-IV,ceftriaxone-IV,cephalexin-PO,cetirizine-PO,chlorothiazide-IV,chlorothiazide-PO,chlorpromazine-PO,chlorthalidone-PO,cholestyramine-PO,ciclosporin-PO,cilostazol-PO,cimetidine-PO,cinacalcet-PO,ciprofloxacin-IV,ciprofloxacin-PO,cisatracurium-IV,citalopram-PO,clarithromycin-PO,clevidipine-IV,clindamycin-IV,clindamycin-PO,clonazepam-PO,clonidine-PO,clopidogrel-PO,codeine-PO,colchicine-PO,colesevelam-PO,cosyntropin-IV,cyanocobalamin (vitamin b12),cyclobenzaprine-PO,dabigatran etexilate-PO,danazol-PO,dapsone-PO,daptomycin-IV,darifenacin-PO,desmopressin-IV,desmopressin-PO,dexamethasone-IV,dexmedetomidine-IV,diatrizoate meglumine-PO,diazepam-PO,dicycloverine-PO,digoxin-IV,digoxin-PO,diltiazem-IV,diltiazem-PO,diphenhydramine-IV,diphenhydramine-PO,dipyridamole-PO,dobutamine-IV,docusate-PO,dofetilide-PO,dolutegravir-PO,donepezil-PO,dopamine-IV,doxazosin-PO,doxepin-PO,doxercalciferol-IV,doxycycline-IV,doxycycline-PO,dronabinol-PO,dronedarone-PO,duloxetine-PO,efavirenz-PO,enalapril-PO,enalaprilat-IV,enoxaparin-SC,epinephrine-IV,"epoetin alfa,recombinant-IV","epoetin alfa,recombinant-SC",epoprostenol-INHALATION,erythromycin-IV,erythromycin-PO,escitalopram oxalate-PO,esmolol-IV,esomeprazole-IV,esomeprazole-PO,estradiol-PO,estrogens-PO,ethacrynate-IV,ethacrynate-PO,etomidate-IV,etravirine-PO,ezetimibe-PO,famotidine-IV,famotidine-PO,febuxostat-PO,felodipine-PO,fenofibrate-PO,fentanyl-IV,fexofenadine-PO,filgrastim-SC,finasteride-PO,fish oil-PO,flecainide-PO,fluconazole-IV,fluconazole-PO,flucytosine-PO,fludrocortisone-PO,fluoxetine-PO,fluticasone-INHALATION,fluticasone-salmeterol,folic acid-IV,folic acid-PO,fondaparinux-SC,formoterol-INHALATION,furosemide-IV,furosemide-PO,gabapentin-PO,galantamine-PO,ganciclovir-IV,gemfibrozil-PO,gentamicin-IV,glimepiride-PO,glipizide-PO,glucose-PO,glyburide / metformin-PO,glyburide-PO,glycopyrronium-IV,guaifenesin,haloperidol-IM,halop

When sorting through the data, be very careful to mind these categories: timedelta, Heart Rhythm, gender, admission_category. These are without numbers. 


In [22]:
# sorting every single id in data files such that we can divide it into 12 hour time chunks:
# i counted the number of IDs in byID: there are 4120 of them! Yay!
# unfortunately there are only 491 dailyID files. I think this may be because there is not enough space. UPDATE: Yep, just checked and there was just not enough space to download all that. 

import os

folder_path = r'C:\Users\faith\OneDrive\Documents\FaithZhang\SURF\RealTimeDetection\Data\byID'

count=0
twelvehrdf= pd.DataFrame()

for IDfile in os.listdir(folder_path):
    file_path = os.path.join(folder_path, IDfile)
    df= pd.read_pickle(file_path)

    new_rows= len(df)//12
    remainder= len(df)%12
    print(len(df), new_rows, remainder)

    

    for chunk in len(range(new_rows)):
        if chunk < new_rows-1:
            null

    if os.path.isfile(file_path):
        count= count +1
print(count)

#mod 12
#for each 12 hours, take the minimum vitals
#for each 12 hours, check the NEXT 12 hours to see if AKI occurs.


49 4 1


DataError: Cannot aggregate non-numeric type: object

Considering 'daily_data' is relatively robust (less affected by inaccurate time indicators)  
We re-arrange the data, using parameters in today, predicting AKI indicator tomorrow (24 hrs ahead)

In [24]:
# prepare training set
training_appended = []
training = pd.DataFrame()

for idx in tqdm(range(len(train_ids))):
    tem_id = train_ids[idx]
    tem_daily = pd.read_pickle( 'C:Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID_daily' + str(tem_id) + '.pkl')
    
    if len(tem_daily) > 2: #for all hadm_ids that represent patients that stayed in  the icu for more than two days
        tem_train = pd.concat(  [tem_daily.iloc[ 0:(len(tem_daily)-2)],  #FROM DAILYIDS: take info for all rows (days) except for the SECOND TO last day and last day
                     pd.DataFrame(tem_daily[ 'AKI_any' ].iloc[ 2:len(tem_daily) ].values) ] #extract the AKI_any column values from the second to last day to end
                    , axis = 1,
                    ).drop( 'AKI_any', axis = 1) #concatenate the second line to the first line such that it is displaced (AKI_any thus becomes AKI_in_24) and the last day is missing a value
        
        training_appended.append(tem_train) #each modified dataframe stored in tem_train and added to training_appended list
        
training = pd.concat(training_appended) #initially empty training df gets all modified dfs added to it.
training.rename( columns = {0:'AKI_in_48'}, inplace = True)
training.to_pickle('train_set_48_faith.pkl')

  0%|          | 0/2884 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID_daily25630745.pkl'

In [5]:
# used a faster way by list to store everything

# # prepare training set
# training = pd.DataFrame()
# for idx in tqdm(range(len(train_ids))):
#     tem_id = train_ids[idx]
#     tem_daily = pd.read_pickle( '../Data/byID_daily/' + str(tem_id) + '.pkl')
    
#     if len(tem_daily) > 1:
#         tem_train = pd.concat(  [tem_daily.iloc[ 0:(len(tem_daily)-1)],  
#                      pd.DataFrame(tem_daily[ 'AKI_any' ].iloc[ 1:len(tem_daily) ].values) ]
#                     , axis = 1,
#                     ).drop( 'AKI_any', axis = 1)
        
#         training = pd.concat( [training, tem_train], ignore_index = True )
        
# training.rename( columns = {0:'AKI_in_24'}, inplace = True)
# training.to_pickle('./train_set.pkl')

In [6]:
# load training if loss connection
training = pd.read_pickle('./train_set.pkl')

testing = pd.DataFrame( columns = np.concatenate( [ ['time'], training.columns.values]))
inform = pd.DataFrame( columns = ['hadm_id', 'if_AKI_inICU','first_AKI_detected'])

testing set will be the rolling window, using ( max(0, t-24), t) to predict (t+1, t+24)  
Record additional information such as  
'if the patients develop AKI during ICU'  
'when is the first time that the patient is found AKI'  
for the future report

CREATING INFORM and TESTING datasets:

In [7]:
testing_appended = []
inform_appended = []

# prepare testing
for idx in tqdm(range(len(test_ids))):
    tem_id = test_ids[idx]
    tem_patient = pd.read_pickle( '../Data/byID/' + str(tem_id) + '.pkl')
    time_stamps = tem_patient['timedelta']
    
    tem_testing = pd.DataFrame( columns = np.concatenate( [ ['time'], training.columns.values]))
    for time_id in range(len(time_stamps)):
        if time_id != (len(time_stamps) - 1):
            tem = pd.DataFrame( data = np.concatenate( (np.array([time_id, tem_id]),
                    tem_patient.loc[ tem_patient['timedelta'].isin( time_stamps[ max(0,(time_id-23)) : (time_id+1)].values ) ].drop( ['hadm_id','timedelta','Heart Rhythm','admission_category'], axis = 1 ).min(axis = 0).values[0:547],
                    tem_patient.loc[ tem_patient['timedelta'].isin( time_stamps[ max(0,(time_id-23)) : (time_id+1)].values ) ].drop( ['hadm_id','timedelta','Heart Rhythm','admission_category'], axis = 1 ).max(axis = 0).values[0:572],
                    np.array( [tem_patient.loc[ tem_patient['timedelta'].isin( time_stamps[ (time_id+1) : min(len(time_stamps), time_id+25 )].values ), 'AKI_any' ].max()])
                    )).reshape( [1,-1] ),
                    columns = testing.columns.values)
            
            testing_appended.append(tem)

    if tem_patient['AKI_any'].max() == 0:
        new_inform = pd.DataFrame( data = np.array([tem_id, tem_patient['AKI_any'].max(), float('NaN')]).reshape([1,-1]),
                    columns = ['hadm_id', 'if_AKI_inICU','first_AKI_detected'])
    else:
        new_inform = pd.DataFrame( data = np.array([tem_id, tem_patient['AKI_any'].max(), tem_patient.loc[ tem_patient['AKI_any'] == 1, 'timedelta'].index[0]]).reshape([1,-1]),
                    columns = ['hadm_id', 'if_AKI_inICU','first_AKI_detected'])
        
    inform_appended.append(new_inform)
    

#testing.to_pickle('./testing/test_set.pkl')
#inform.to_pickle('./testing/inform.pkl')

100%|██████████| 1236/1236 [27:03<00:00,  1.31s/it] 


In [8]:
testing = pd.concat(testing_appended)
inform = pd.concat(inform_appended)

In [9]:
testing.loc[testing['gender'] == 'M', 'gender'] = 0
testing.loc[testing['gender'] == 'F', 'gender'] = 1 

In [10]:
testing = testing.astype( 'float' )
testing.to_pickle('./test_set.pkl')

In [11]:
inform.to_pickle('./inform.pkl')